# 스팸 문자를 Naive Bayes를 이용해 분류하기

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(2021)

## 1. Data
### 1.1 Data Load
`sms_spam.csv` 데이터는 문자 내용이 스팸인지 아닌지를 구분하기 위한 데이터

In [10]:
spam = pd.read_csv('/content/drive/MyDrive/MLStudy/ML Algorithm/05_Naive_Bayes/sms_spam.csv')

In [25]:
text = spam["text"]
label = spam["type"]

### 1.2 Data EDA

In [26]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [27]:
label[0]

'ham'

In [28]:
label.value_counts()

ham     4827
spam     747
Name: type, dtype: int64

### 1.3 Data Cleaning
정답의 문자를 숫자로 변환시키기
ham은 0으로, spam은 1로 변환

In [29]:
label = label.map({"ham": 0, "spam": 1})

In [30]:
label.value_counts()

0    4827
1     747
Name: type, dtype: int64

text를 문자만 존재하도록 정리하기
regex를 통해 영어, 숫자 그리고 띄어쓰기를 제외한 모든 단어를 지우기

In [31]:
re_pattern = "[^a-zA-Z0-9\ ]"

In [32]:
text[0]

'Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...'

In [33]:
text.iloc[:1].str.replace(re_pattern, "", regex=True)[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [34]:
text = text.str.replace(re_pattern, "", regex=True)

대문자들을 모두 소문자로 바꿔주기

In [35]:
text[0]

'Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat'

In [36]:
text.iloc[:1].str.lower()[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

In [37]:
text = text.str.lower()

In [38]:
text[0]

'go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat'

### 1.4 Data Split

In [39]:
from sklearn.model_selection import train_test_split

train_text, test_text, train_label, test_label = train_test_split(
    text, label, train_size=0.7, random_state=2021
)

In [41]:
print(f"train_data_size: {len(train_label)}, {len(train_label)/len(text):.2f}")
print(f"test_data_size: {len(test_label)}, {len(test_label)/len(text):.2f}")

train_data_size: 3901, 0.70
test_data_size: 1673, 0.30


## 2. Count Vectorize\
Naive Bayes를 학습시키기 위해서 각 문장에서 단어들이 몇 번 나왔는 지로 변환

### 2.1 word tokenize
문장을 단어로 나누는 데에는 `nltk` 패키지의 `word_tokenize`를 이용함

In [42]:
import nltk
from nltk import word_tokenize

nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [43]:
train_text.iloc[0]

'am only searching for good dual sim mobile pa'

In [44]:
word_tokenize(train_text.iloc[0])

['am', 'only', 'searching', 'for', 'good', 'dual', 'sim', 'mobile', 'pa']

### 2.2 count vectorize
`sklearn.feature_extraction.text`의 `CountVectorizer`를 이용해 단어들을 count vector로 만들기

In [46]:
from sklearn.feature_extraction.text import CountVectorizer

예시로 2개의 문장으로 CountVectorizer 학습

In [47]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [48]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)

In [49]:
cnt_vectorizer.fit(train_text.iloc[:2])

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7fbbbf150310>)

문장에서 나온 단어들은 다음과 같음

In [50]:
cnt_vectorizer.vocabulary_

{'am': 0,
 'only': 8,
 'searching': 12,
 'for': 4,
 'good': 5,
 'dual': 2,
 'sim': 14,
 'mobile': 7,
 'pa': 9,
 'excellent': 3,
 'ill': 6,
 'see': 13,
 'what': 15,
 'rileys': 11,
 'plans': 10,
 'are': 1}

In [51]:
vocab = sorted(cnt_vectorizer.vocabulary_.items(), key=lambda x: x[1])
vocab = list(map(lambda x: x[0], vocab))
vocab

['am',
 'are',
 'dual',
 'excellent',
 'for',
 'good',
 'ill',
 'mobile',
 'only',
 'pa',
 'plans',
 'rileys',
 'searching',
 'see',
 'sim',
 'what']

In [52]:
sample_cnt_vector = cnt_vectorizer.transform(train_text.iloc[:2]).toarray()
sample_cnt_vector

array([[1, 0, 1, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 1, 0],
       [0, 1, 0, 1, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 1]])

In [53]:
train_text.iloc[:2].values

array(['am only searching for good dual sim mobile pa',
       'excellent ill see what rileys plans are'], dtype=object)

In [54]:
pd.DataFrame(sample_cnt_vector, columns=vocab)

,am,are,dual,excellent,for,good,ill,mobile,only,pa,plans,rileys,searching,see,sim,what
0,1,0,1,0,1,1,0,1,1,1,0,0,1,0,1,0
1,0,1,0,1,0,0,1,0,0,0,1,1,0,1,0,1


문장에서 각 단어가 나온 횟수를 계산해주는 것이 counter vectorize의 역할

#### 2.2.1 학습
모든 데이터에 대해 count vectorize 진행

In [56]:
cnt_vectorizer = CountVectorizer(tokenizer=word_tokenize)
cnt_vectorizer.fit(train_text)

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CountVectorizer(tokenizer=<function word_tokenize at 0x7fbbbf150310>)

전체 단어는 총 7908개

In [57]:
len(cnt_vectorizer.vocabulary_)

7908

#### 2.2.2 예측

In [58]:
train_matrix = cnt_vectorizer.transform(train_text)
test_matrix = cnt_vectorizer.transform(test_text)

CountVectorize는 학습한 단어장에 존재하지 않는 단어가 들어오는 경우 무시함

In [59]:
cnt_vectorizer.transform(["notavsleiwheitd"]).toarray().sum()

0

## 3. Naive Bayes
분류를 위한 Naive Bayes 모델은 `sklearn.naive_bayes`의 `BernoulliNB`를 사용

In [60]:
from sklearn.naive_bayes import BernoulliNB
naive_bayes = BernoulliNB()

### 3.1 학습

In [61]:
naive_bayes.fit(train_matrix, train_label)

BernoulliNB()

### 3.2 예측

In [62]:
train_pred = naive_bayes.predict(train_matrix)
test_pred = naive_bayes.predict(test_matrix)

### 3.3 평가

In [63]:
from sklearn.metrics import accuracy_score

train_acc = accuracy_score(train_label, train_pred)
test_acc = accuracy_score(test_label, test_pred)

In [64]:
print(f"Train Accuracy is {train_acc:.4f}")
print(f"Test Accuracy is {test_acc:.4f}")

Train Accuracy is 0.9854
Test Accuracy is 0.9767
